In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sun Aug 29 12:48:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    27W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import BatchNormalization
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras import initializers

In [ ]:
import cv2

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Training data/train.csv')

In [ ]:
train.head()

,Unnamed: 0,filename,emotion,durations
0,0,18777.mp3,neutral,2.44000
1,1,24041.mp3,neutral,3.36000
2,2,1621.mp3,joy,0.34068
3,3,28883.mp3,neutral,1.56000
4,4,14918.mp3,neutral,2.82000


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Training data/test.csv')

In [ ]:
emotion_dict = {}
i = 0
for emotion in train['emotion'].unique():
    emotion_dict[emotion] = i
    i += 1

Y = train['emotion'].apply(lambda x: emotion_dict[x]).to_numpy()

In [ ]:
Y

array([0, 0, 1, ..., 0, 3, 0])

In [ ]:
X = np.load('/content/drive/MyDrive/Training data/traindata5.npy')

In [ ]:
X = X[..., np.newaxis]

In [ ]:
X.shape

(5816, 862, 20, 1)

In [ ]:
sub = np.load('/content/drive/MyDrive/Training data/testdata5.npy')

In [ ]:
sub = sub[..., np.newaxis]

In [ ]:
sub.shape

(2492, 862, 20, 1)

In [ ]:
mean = np.mean(np.concatenate((X,sub)), axis=0)
std = np.std(np.concatenate((X,sub)), axis=0)

X = (X-mean)/std
sub = (sub - mean)/std

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)

In [ ]:
X_train.shape

(5234, 862, 20, 1)

In [ ]:
def build_model(input_shape):
    model = keras.Sequential()

    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    model.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(keras.layers.Conv2D(254, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1000, activation='relu'))
    model.add(keras.layers.Dense(200, activation='relu'))
    model.add(keras.layers.Dense(7, activation='softmax'))

    return model

In [ ]:
input_shape = (X_train.shape[1], X_train.shape[2],1)
model = build_model(input_shape)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 860, 18, 64)       640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 858, 16, 32)       18464     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 429, 8, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 429, 8, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 427, 6, 128)       36992     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 425, 4, 254)       292862    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 213, 2, 254)       0

In [ ]:
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=30)

164/164 [==============================] - 27s 163ms/step - loss: 0.1265 - accuracy: 0.9595 - val_loss: 2.5820 - val_accuracy: 0.5223


In [ ]:
ypred = model.predict(sub)
pred = np.argmax(ypred,axis=1)

In [ ]:
pred

array([0, 0, 0, ..., 6, 5, 0])

In [ ]:
test

,Unnamed: 0,filename,durations
0,0,26199.mp3,0.680272
1,1,692.mp3,2.690023
2,2,16821.mp3,1.310023
3,3,1608.mp3,1.650023
4,4,47947.wav,3.870567
...,...,...,...
2487,2487,16692.mp3,1.060000
2488,2488,29505.mp3,6.280000
2489,2489,5807.mp3,2.150023
2490,2490,43982.wav,2.268980


In [ ]:
submission_file = pd.DataFrame()
submission_file['filename'] = test['filename']
inv_emotion_dict = {v: k for k, v in emotion_dict.items()}

pred_emotion = np.array([inv_emotion_dict[i] for i in pred])

In [ ]:
submission_file['emotion'] = pred_emotion

In [ ]:
submission_file

,filename,emotion
0,26199.mp3,neutral
1,692.mp3,neutral
2,16821.mp3,neutral
3,1608.mp3,neutral
4,47947.wav,disgust
...,...,...
2487,16692.mp3,neutral
2488,29505.mp3,neutral
2489,5807.mp3,anger
2490,43982.wav,fear


In [ ]:
submission_file.to_csv('submission.csv', index=False)